In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score, classification_report
import pickle
import os
import glob
import seaborn as sns
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn import preprocessing
from sklearn.utils import class_weight
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

In [3]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [11]:
result = pd.read_excel('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/client_data/predict/sure data.xlsx')

In [12]:
result.head()

,,L*,a*,b*,X,Y,Z,Dominant Wavelenght,Whiteness,Purity,...,978.616,979.335,980.054,980.772,981.491,982.209,982.928,983.646,984.364,985.082
0,W_Calibration,100.0000,0.000000,0.00000,95.0422,100.0000,108.8760,565.016,99.9909,0.000017,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.000,100.0000,100.0000,100.0000
1,F_Calibration,100.0000,0.000000,0.00000,108.1040,100.0000,39.2965,491.070,99.9909,0.000170,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.000,100.0000,100.0000,100.0000
2,W_GIA_ROUND_H_None_1.18_VS2,99.8384,-1.144380,2.19537,93.9961,99.5825,104.8850,567.764,89.9613,0.019141,...,103.9380,103.6940,103.7560,103.6450,103.5940,103.0600,129.454,-55.8714,533.6070,-46.2620
3,F_GIA_ROUND_H_None_1.18_VS2,98.5934,1.491540,5.90488,105.1660,96.4062,34.5881,584.804,72.0208,0.077836,...,79.8317,80.1702,78.7726,77.3831,75.8332,74.1515,-109.600,-52.9897,28.2192,173.8030
4,W_GIA_PEAR_E_None_0.59_VVS1,98.2432,-0.374826,1.28889,90.5820,95.5251,101.9750,571.685,89.8094,0.011950,...,95.2066,95.1769,95.1780,95.1140,95.2294,94.9125,157.731,78.5566,400.3560,72.7357


In [13]:
result.rename(columns = {' ': 'Output'}, inplace=True)

In [16]:
result.head()

,Output,L*,a*,b*,X,Y,Z,Dominant Wavelenght,Whiteness,Purity,...,978.616,979.335,980.054,980.772,981.491,982.209,982.928,983.646,984.364,985.082
0,W_Calibration,100.0000,0.000000,0.00000,95.0422,100.0000,108.8760,565.016,99.9909,0.000017,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.000,100.0000,100.0000,100.0000
1,F_Calibration,100.0000,0.000000,0.00000,108.1040,100.0000,39.2965,491.070,99.9909,0.000170,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.000,100.0000,100.0000,100.0000
2,W_GIA_ROUND_H_None_1.18_VS2,99.8384,-1.144380,2.19537,93.9961,99.5825,104.8850,567.764,89.9613,0.019141,...,103.9380,103.6940,103.7560,103.6450,103.5940,103.0600,129.454,-55.8714,533.6070,-46.2620
3,F_GIA_ROUND_H_None_1.18_VS2,98.5934,1.491540,5.90488,105.1660,96.4062,34.5881,584.804,72.0208,0.077836,...,79.8317,80.1702,78.7726,77.3831,75.8332,74.1515,-109.600,-52.9897,28.2192,173.8030
4,W_GIA_PEAR_E_None_0.59_VVS1,98.2432,-0.374826,1.28889,90.5820,95.5251,101.9750,571.685,89.8094,0.011950,...,95.2066,95.1769,95.1780,95.1140,95.2294,94.9125,157.731,78.5566,400.3560,72.7357


In [18]:
result_2= result.loc[:,300.377:700.577]
# df_without_calb_None['414.985':'537.294']

In [19]:
result_3 =result.loc[:,'Output']
result_2= result.loc[:,300.377:700.577]
result_1= result.iloc[:,1:14]


In [20]:
print(result_2.shape, result_3.shape)

(344, 523) (344,)


In [21]:
df = pd.concat([result_2,result_3], axis=1)
df.head()


,300.377,301.162,301.946,302.731,303.515,304.299,305.083,305.867,306.652,307.435,...,694.59,695.339,696.087,696.836,697.584,698.332,699.081,699.829,700.577,Output
0,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,W_Calibration
1,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,F_Calibration
2,94.2414,94.3561,94.2523,94.4179,94.6916,94.7946,94.5172,94.6970,94.3413,94.3115,...,98.7710,98.7736,98.7744,98.7726,98.7838,98.7563,98.7442,98.7256,98.7439,W_GIA_ROUND_H_None_1.18_VS2
3,74.0615,73.4640,73.4925,73.0328,72.5936,72.5789,72.1029,72.4686,72.8165,72.9914,...,104.9700,104.6670,105.2850,105.6830,106.0880,105.9440,105.1730,105.0920,105.6360,F_GIA_ROUND_H_None_1.18_VS2
4,92.6120,92.5131,92.5559,92.5364,93.3298,93.2499,92.8878,93.6046,93.8392,93.7318,...,96.0486,96.0498,96.0770,96.0704,96.0929,96.0851,96.0755,96.0833,96.0994,W_GIA_PEAR_E_None_0.59_VVS1


In [10]:
# df.to_excel('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/Processed_data/whole_data.xlsx')

In [91]:
df_without_calb = df[~df['Output'].str.contains(('Calibration|CALIBARTION|CALIBARATION'),case=False, regex=True, na = False)]
df_without_calb.reset_index()
df_without_calb['w/f']= df_without_calb['Output'].str.split('_').str[0]

In [92]:
df_without_calb.shape

(232, 525)

In [264]:
df_with_w = df_without_calb[df_without_calb['w/f'] == 'W']

In [211]:
df_with_w.shape

(116, 525)

In [265]:
df_with_w['quality'] = df_with_w['Output'].str.split('_').str[4]
df_with_w['quality'].replace({'1.01':'Faint','3.01':'None','1.50':'Medium','lb':'Faint','Benth':'None','brown':'None','Pink':'None', 'LB':'None','LB ':'None', 'Faint YELLOW':'Faint', 'Faint YELLOW':'Faint', 'Faint GREEN':'Faint','  ':'None', 'BROWN':'Faint', 'Lblack':'Faint', 'lb ':'None','Black':'None'}, inplace = True)

In [266]:
df_with_w['color'] = df_with_w['Output'].str.split('_').str[3]
df_with_w['color'].replace({'Medium':'D','Faint':'I','None':'K','I/LB':'I','J ':'J','F/LB':'F','G/LB':'G','E LB':'E', ' K':'K', 'G LB ':'G', 'H ':'H'}, inplace = True)


In [79]:
df_with_w.drop(columns=['Output','w/f'], inplace=True)

In [102]:
df_with_w.drop(index=[1043,1049],inplace = True)

In [267]:
df_with_w['quality'].replace({'None':0, 'Faint': 1, 'Very Strong':3, 'Medium':2, 'Strong':4}, inplace=True)

In [268]:
df_with_w['quality'].value_counts()

     87
0    20
1     5
2     4
Name: quality, dtype: int64

In [269]:
df_with_w_none = df_with_w[(df_with_w['quality'] != '')&(df_with_w['quality'] == 0)]

In [274]:
df_with_w_none['color'].value_counts()

I    5
H    4
E    4
D    4
J    2
K    1
Name: color, dtype: int64

In [218]:
df_with_w_ = df_with_w[(df_with_w['quality'] == '')]

In [219]:
df_with_w_

,300.377,301.162,301.946,302.731,303.515,304.299,305.083,305.867,306.652,307.435,...,696.836,697.584,698.332,699.081,699.829,700.577,Output,w/f,quality,color
74,96.9786,96.6674,96.5518,96.2488,96.3056,96.8856,96.9967,96.9900,97.2296,96.8493,...,100.3690,100.3770,100.3670,100.3990,100.4210,100.4300,W_GIA_EMERALD_E__0.800_VVS1,W,,E
76,88.6644,88.6419,88.6595,88.8186,89.2789,89.5550,89.6877,89.6706,89.6673,89.5574,...,97.6869,97.6991,97.6944,97.7283,97.7522,97.7642,W_GIA_EMERALD_H__0.800_VVS1,W,,H
80,92.5871,92.4423,92.4396,92.9280,93.1816,93.4024,93.2139,93.2396,93.5883,93.8788,...,97.7574,97.7496,97.7586,97.7500,97.7457,97.7632,W_GIA_EMERALD_F__0.720_VVS2,W,,F
82,92.2041,92.3659,92.5884,92.9841,92.7361,93.0234,93.1909,92.9419,92.9119,93.1650,...,100.9900,101.0070,101.0320,101.0310,101.0450,101.0080,W_GIA_HEART_G__0.800_FL/IF,W,,G
84,75.2969,75.4791,75.3742,75.7334,75.6110,75.7123,75.4899,75.7400,76.2423,76.4867,...,77.4116,77.4161,77.4361,77.4519,77.4360,77.4407,W_GIA_HEART_E__0.700_VVS2,W,,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,98.8213,98.8940,99.2403,98.8460,98.8773,98.9235,98.9834,98.5393,98.5183,99.0709,...,106.0220,106.0370,106.0200,106.0680,106.0590,106.0420,W_GIA_PRINCESS_G__0.800_VVS2,W,,G
310,98.7311,98.7766,98.5919,98.2787,98.0296,97.7440,97.6028,98.1065,98.2342,98.3290,...,102.2430,102.2550,102.2050,102.2100,102.1910,102.2100,W_GIA_PRINCESS_F__0.700_VVS2,W,,F
312,97.1921,96.8519,96.5520,96.2237,96.1270,96.0212,96.2022,96.4929,96.5884,96.8306,...,103.5400,103.5650,103.4960,103.5070,103.5180,103.5530,W_GIA_PRINCESS_H__0.710_VVS2,W,,H
316,104.7010,104.3720,104.2610,103.6610,103.6420,103.8180,103.9630,103.6780,103.8070,103.7800,...,109.6540,109.6370,109.6450,109.6430,109.6760,109.7210,W_GIA_PRINCESS_G__0.820_VVS1,W,,G


In [220]:
X = df_with_w_new.iloc[:,:-4]

In [223]:
pickle_model = pickle.load(open('/home/pankaj_v/Documents/Data_analysis_dixit/Models/quality_random_forest_model.pkl','rb'))

In [224]:
Y = pickle_model.predict(X)

In [225]:
df_with_w_['quality'] = Y

In [226]:
df_with_w = pd.concat([df_with_w_,df_with_w[(df_with_w['quality'] != '')]])

In [234]:
df_with_w['quality'].replace({'None':0, 'Faint': 1, 'Very Strong':3, 'Medium':2, 'Strong':4}, inplace=True)

In [236]:
df_with_w.drop(columns=['Output','w/f'], inplace=True)

In [248]:
# df for nonedata1[data1['color'] == 'K']
df_with_w_none= df_with_w[df_with_w['quality'] == 0]

In [271]:
X = df_with_w_none.loc[:,363.707:423.503]
Y = df_with_w_none.iloc[:,-1:]

In [276]:
X.head()

,363.707,364.486,365.266,366.044,366.823,367.602,368.381,369.160,369.938,370.717,...,416.535,417.309,418.084,418.858,419.632,420.407,421.181,421.955,422.729,423.503
2,89.3927,89.2308,89.1855,89.2610,89.0716,89.0472,88.9181,88.8815,88.9155,89.1525,...,84.9225,85.8822,87.2227,88.9566,90.4205,91.3557,91.9758,92.3921,92.7691,93.0904
4,91.5949,91.5384,90.9536,90.9776,90.8458,90.5510,90.4396,90.4709,90.6611,90.8047,...,88.9238,89.3151,89.8706,90.5888,91.1822,91.5591,91.7775,91.9360,92.1483,92.3207
8,82.6837,82.7789,82.3052,82.2172,81.6174,81.5266,81.1751,80.4829,80.0653,79.6332,...,72.3909,74.1369,76.4631,79.3677,81.9347,83.6382,84.8184,85.6573,86.4115,87.0172
12,92.7166,92.3915,92.2116,92.4628,92.3681,92.4716,92.4391,91.9754,92.0643,91.9234,...,89.7023,90.0417,90.5746,91.2639,91.8392,92.1450,92.3061,92.4812,92.5911,92.7043
16,101.4130,101.3520,101.5630,101.3260,101.1470,100.8680,100.8770,100.8720,100.9960,101.1850,...,101.2600,101.3450,101.3770,101.3200,101.3400,101.3220,101.3380,101.3370,101.3260,101.3230


In [277]:
principal=PCA(n_components=5)
X_test = principal.fit_transform(X)

In [278]:
pickle_model = pickle.load(open('/home/pankaj_v/Documents/Data_analysis_dixit/Models/et_none_model.pkl','rb'))

In [279]:
y_pred = pickle_model.predict(X_test)

In [280]:
print(classification_report(Y,y_pred))

              precision    recall  f1-score   support

           D       0.50      0.25      0.33         4
           E       0.00      0.00      0.00         4
           F       0.00      0.00      0.00         0
           H       0.00      0.00      0.00         4
           I       0.14      0.20      0.17         5
           J       1.00      0.50      0.67         2
           K       0.00      0.00      0.00         1

    accuracy                           0.15        20
   macro avg       0.23      0.14      0.17        20
weighted avg       0.24      0.15      0.17        20

